In [ ]:
!pip install datasets transformers
!apt install git-lfs
!pip install sentence_transformers
!pip install gradio
!pip install transformers
import gradio as gr

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',  force_remount=True)

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import json
import transformers
from datasets import load_dataset, load_metric
import ast
from sentence_transformers import SentenceTransformer, util
print(transformers.__version__)

4.27.4


In [ ]:
import logging
import os.path
import zipfile
import json
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import torch
from transformers import AutoTokenizer
from transformers import AutoModelForQuestionAnswering

In [ ]:
%cd "/content/gdrive/MyDrive"

/content/gdrive/MyDrive


## Input the question

In [ ]:
question = input('What is the question: ')
file = input('Input the file you want to use: ')

What is the question: How many pilots volunteer to join the Wingman project?
Input the file you want to use: aa.json


## Extract text from json

In [ ]:
def extract_text_from_json(file):
    with open('/content/gdrive/MyDrive/' + file, "r") as json_file:
        y = json.load(json_file)
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    out = pd.DataFrame(out.items())
    out.columns = ['Key', 'Content']
    df = out[out['Key'].str.endswith('Text')]
    return df

In [ ]:
extract_text_from_json(file)

,Key,Content
81,elements_3_Text,About This Report
103,elements_4_Text,American Airlines is committed to providing re...
123,elements_5_Text,Task Force on Climate-
142,elements_6_Text,related Financial Disclosures (TCFD)
161,elements_7_Text,Sustainability Accounting
...,...,...
97099,elements_3877_Text,"Nicole Muñoz, Vice President - ECS Environment..."
97119,elements_3878_Text,"SCS Global Services, 2000 Powell Street, Suite..."
97139,elements_3879_Text,"Emeryville, CA 94608 USA"
97190,elements_3882_Text,Forward-Looking Statements


## Calculate relevancy score

In [ ]:
def semantic_score(data, question, context_column):
    embedder = SentenceTransformer('all-MiniLM-L6-v2')
    
    new_data = data.dropna(subset = [context_column])
    corpus = new_data[context_column].tolist()
    corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)
    queries = [question]
    top_k = min(3, len(corpus))
    for query in queries:
        query_embedding = embedder.encode(query, convert_to_tensor=True)

        # We use cosine-similarity and torch.topk to find the highest 3 scores
        cos_scores = util.cos_sim(query_embedding, corpus_embeddings)[0]
        top_results = torch.topk(cos_scores, k=top_k)
        
        score_list = []
        context_list = []
        for score, idx in zip(top_results[0], top_results[1]):
            score_list.append(float(score))
            context_list.append(corpus[idx])
    
    result = {'context': context_list, 'score': score_list}
    result = pd.DataFrame(result)
    result = ' '.join(result['context'])
    return result

## Format the test dataset

In [ ]:
file_used = extract_text_from_json(file)
context = semantic_score(file_used, question, 'Content')
title = 'sample'

In [ ]:
test_data = {'id': [1],
        'title': [title],
        'context': [context],
        'question': [question],
        'answers': [{'answer_start': [-1], 'text': ['No answer exists']}]      }

test_data = pd.DataFrame(test_data)
test_data.to_csv('/content/gdrive/MyDrive/test_data.csv', index = False, sep = ',')

In [ ]:
from datasets import load_dataset
datasets = load_dataset('csv', data_files={"train":'/content/gdrive/MyDrive/train_data.csv', "validation": '/content/gdrive/MyDrive/test_data.csv'})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-a4fbdcd511bfb67e/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
squad_v2 = False
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

In [ ]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [ ]:
max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

In [ ]:
for i, example in enumerate(datasets["train"]):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > 384:
        break
example = datasets["train"][i]

In [ ]:
example

{'id': 81,
 'title': 'giving',
 'context': "The ISPCC directly provides a range of services to children and families, and advocates for change to enhance the lives of children in Ireland. Ryanair commenced its partnership with ISPCC Childline five years ago and has fundraised more than €500,000 to date. Over the years, we have engaged in a number of initiatives to support the work of the charity, ensuring that Childline can continue listening to children 24 hours a day, every day. The ISPCC and Ryanair work in collaboration on a number of campaigns and competitions to help make the protection of children everybody’s priority. €500k in 5 years raised for the ISPCC. Continuation of ISPCC’s essential Childline services is now also online thanks to Ryanair funds. We have worked with Pequeño Deseo for more than five years, first as one of our scratch card charity partners and since 2019, as chosen European charity partner. We believe in giving back to organisations that need further funding

In [ ]:
len(tokenizer(example["question"], example["context"])["input_ids"])

427

In [ ]:
len(tokenizer(example["question"], example["context"], max_length=max_length, truncation="only_second")["input_ids"])

384

In [ ]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)

In [ ]:
[len(x) for x in tokenized_example["input_ids"]]

[384, 183]

In [ ]:
for x in tokenized_example["input_ids"][:2]:
    print(tokenizer.decode(x))

[CLS] how much is ( was ) global giving? [SEP] the ispcc directly provides a range of services to children and families, and advocates for change to enhance the lives of children in ireland. ryanair commenced its partnership with ispcc childline five years ago and has fundraised more than €500, 000 to date. over the years, we have engaged in a number of initiatives to support the work of the charity, ensuring that childline can continue listening to children 24 hours a day, every day. the ispcc and ryanair work in collaboration on a number of campaigns and competitions to help make the protection of children everybody ’ s priority. €500k in 5 years raised for the ispcc. continuation of ispcc ’ s essential childline services is now also online thanks to ryanair funds. we have worked with pequeno deseo for more than five years, first as one of our scratch card charity partners and since 2019, as chosen european charity partner. we believe in giving back to organisations that need further

In [ ]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)
print(tokenized_example["offset_mapping"][0][:100])

[(0, 0), (0, 3), (4, 8), (9, 11), (12, 13), (13, 16), (16, 17), (18, 24), (25, 31), (31, 32), (0, 0), (0, 3), (4, 6), (6, 8), (8, 9), (10, 18), (19, 27), (28, 29), (30, 35), (36, 38), (39, 47), (48, 50), (51, 59), (60, 63), (64, 72), (72, 73), (74, 77), (78, 87), (88, 91), (92, 98), (99, 101), (102, 109), (110, 113), (114, 119), (120, 122), (123, 131), (132, 134), (135, 142), (142, 143), (144, 148), (148, 151), (152, 161), (162, 165), (166, 177), (178, 182), (183, 185), (185, 187), (187, 188), (189, 194), (194, 198), (199, 203), (204, 209), (210, 213), (214, 217), (218, 221), (222, 226), (226, 229), (229, 232), (233, 237), (238, 242), (243, 244), (244, 247), (247, 248), (248, 251), (252, 254), (255, 259), (259, 260), (261, 265), (266, 269), (270, 275), (275, 276), (277, 279), (280, 284), (285, 292), (293, 295), (296, 297), (298, 304), (305, 307), (308, 319), (320, 322), (323, 330), (331, 334), (335, 339), (340, 342), (343, 346), (347, 354), (354, 355), (356, 364), (365, 369), (370, 375

In [ ]:
first_token_id = tokenized_example["input_ids"][0][1]
offsets = tokenized_example["offset_mapping"][0][1]
print(tokenizer.convert_ids_to_tokens([first_token_id])[0], example["question"][offsets[0]:offsets[1]])

how how


In [ ]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 397
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 1
    })
})

In [ ]:
example

{'id': 81,
 'title': 'giving',
 'context': "The ISPCC directly provides a range of services to children and families, and advocates for change to enhance the lives of children in Ireland. Ryanair commenced its partnership with ISPCC Childline five years ago and has fundraised more than €500,000 to date. Over the years, we have engaged in a number of initiatives to support the work of the charity, ensuring that Childline can continue listening to children 24 hours a day, every day. The ISPCC and Ryanair work in collaboration on a number of campaigns and competitions to help make the protection of children everybody’s priority. €500k in 5 years raised for the ISPCC. Continuation of ISPCC’s essential Childline services is now also online thanks to Ryanair funds. We have worked with Pequeño Deseo for more than five years, first as one of our scratch card charity partners and since 2019, as chosen European charity partner. We believe in giving back to organisations that need further funding

In [ ]:
#debugged
import ast
answers = example["answers"]
answers = ast.literal_eval(answers)
#print(type(answers))
#start_char = answers["answer_start"][0]
#start_char = answers["answer_start"][0]
#print(start_char)
#print(len(answers["text"][0]))

In [ ]:
answers = example["answers"]
answers = ast.literal_eval(answers)
start_char = answers["answer_start"][0]
end_char = start_char + len(answers["text"][0])

# Start token index of the current span in the text.
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

# End token index of the current span in the text.
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

# Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
offsets = tokenized_example["offset_mapping"][0]
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    # Move the token_start_index and token_end_index to the two ends of the answer.
    # Note: we could go after the last offset if the answer is the last word (edge case).
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print(start_position, end_position)
else:
    print("The answer is not in this feature.")

133 144


In [ ]:
print(tokenizer.decode(tokenized_example["input_ids"][0][start_char: end_char+1]))
print(answers["text"][0])


€500k in 5 years raised for the ISPCC


In [ ]:
pad_on_right = tokenizer.padding_side == "right"

In [ ]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        answers = ast.literal_eval(answers)#ADDING
        if answers["answer_start"] == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = int(start_char) + int(len(answers["text"][0])) #CHANGEDDDD

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [ ]:
features = prepare_train_features(datasets['train'][:1])#:5
#features

In [ ]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

Map:   0%|          | 0/397 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
#model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
model = AutoModelForQuestionAnswering.from_pretrained('/content/gdrive/MyDrive/my_model', local_files_only=True)

In [ ]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-squad",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=50,
    weight_decay=0.01,
    push_to_hub=True,
    logging_steps = 1,
    #output_dir = 'JoJoWeixiChen/AB-Finetuned-model',
    save_strategy = 'no',
    metric_for_best_model='accuracy',
)

In [ ]:
'''model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-squad",
    #evaluation_strategy = "epoch",
    #logging_strategy = "epoch",
    #save_strategy = 'epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    eval_steps = 100, 
    num_train_epochs=50,
    logging_steps = 1,
    weight_decay=0.01,
    #load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    push_to_hub=True,
)'''

'model_name = model_checkpoint.split("/")[-1]\nargs = TrainingArguments(\n    f"{model_name}-finetuned-squad",\n    #evaluation_strategy = "epoch",\n    #logging_strategy = "epoch",\n    #save_strategy = \'epoch\',\n    learning_rate=2e-5,\n    per_device_train_batch_size=batch_size,\n    per_device_eval_batch_size=batch_size,\n    eval_steps = 100, \n    num_train_epochs=50,\n    logging_steps = 1,\n    weight_decay=0.01,\n    #load_best_model_at_end=True,\n    metric_for_best_model=\'accuracy\',\n    push_to_hub=True,\n)'

In [ ]:
from transformers import default_data_collator

data_collator = default_data_collator

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
'''def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    #f1 = f1_score(labels, predictions, average = "weighted")#average = "weighted"
    acc = accuracy_score(predictions, labels)
    return {"accuracy" : acc }#,"f1" : f1}'''

'def compute_metrics(eval_pred):\n    predictions, labels = eval_pred\n    predictions = np.argmax(predictions, axis=1)\n    #f1 = f1_score(labels, predictions, average = "weighted")#average = "weighted"\n    acc = accuracy_score(predictions, labels)\n    return {"accuracy" : acc }#,"f1" : f1}'

In [ ]:
'''from datasets import load_metric
metric = load_metric('accuracy')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=0)
    return metric.compute(predictions=predictions, references=labels)'''

"from datasets import load_metric\nmetric = load_metric('accuracy')\n\ndef compute_metrics(eval_pred):\n    predictions, labels = eval_pred\n    predictions = np.argmax(predictions, axis=0)\n    return metric.compute(predictions=predictions, references=labels)"

In [ ]:
'''def compute_metrics(eval_pred):
    metrics = ["accuracy", "recall", "precision", "f1"] #List of metrics to return
    metric={}
    for met in metrics:
       metric[met] = load_metric(met)
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    metric_res={}
    for met in metrics:
       metric_res[met]=metric[met].compute(predictions=predictions, references=labels)[met]
    return metric_res'''

'def compute_metrics(eval_pred):\n    metrics = ["accuracy", "recall", "precision", "f1"] #List of metrics to return\n    metric={}\n    for met in metrics:\n       metric[met] = load_metric(met)\n    logits, labels = eval_pred\n    predictions = np.argmax(logits, axis=-1)\n    metric_res={}\n    for met in metrics:\n       metric_res[met]=metric[met].compute(predictions=predictions, references=labels)[met]\n    return metric_res'

In [ ]:
'''trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer
    #callbacks=[CombinedTensorBoardCallback]
    #compute_metrics=compute_metrics
)'''

'trainer = Trainer(\n    model,\n    args,\n    train_dataset=tokenized_datasets["train"],\n    eval_dataset=tokenized_datasets["validation"],\n    data_collator=data_collator,\n    tokenizer=tokenizer\n    #callbacks=[CombinedTensorBoardCallback]\n    #compute_metrics=compute_metrics\n)'

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer
)

/content/gdrive/MyDrive/distilbert-base-uncased-finetuned-squad is already a clone of https://huggingface.co/JoJoWeixiChen/distilbert-base-uncased-finetuned-squad. Make sure you pull the latest changes with `repo.git_pull()`.


OSError: ignored

In [ ]:
import os
from transformers.integrations import TrainerCallback, is_tensorboard_available

def custom_rewrite_logs(d, mode):
    new_d = {}
    eval_prefix = "eval_"
    eval_prefix_len = len(eval_prefix)
    test_prefix = "test_"
    test_prefix_len = len(test_prefix)
    for k, v in d.items():
        if mode == 'eval' and k.startswith(eval_prefix):
            if k[eval_prefix_len:] == 'loss':
                new_d["combined/" + k[eval_prefix_len:]] = v
        elif mode == 'test' and k.startswith(test_prefix):
            if k[test_prefix_len:] == 'loss':
                new_d["combined/" + k[test_prefix_len:]] = v
        elif mode == 'train':
            if k == 'loss':
                new_d["combined/" + k] = v
    return new_d


class CombinedTensorBoardCallback(TrainerCallback):
    """
    A [`TrainerCallback`] that sends the logs to [TensorBoard](https://www.tensorflow.org/tensorboard).
    Args:
        tb_writer (`SummaryWriter`, *optional*):
            The writer to use. Will instantiate one if not set.
    """

    def __init__(self, tb_writers=None):
        has_tensorboard = is_tensorboard_available()
        if not has_tensorboard:
            raise RuntimeError(
                "TensorBoardCallback requires tensorboard to be installed. Either update your PyTorch version or"
                " install tensorboardX."
            )
        if has_tensorboard:
            try:
                from torch.utils.tensorboard import SummaryWriter  # noqa: F401

                self._SummaryWriter = SummaryWriter
            except ImportError:
                try:
                    from tensorboardX import SummaryWriter

                    self._SummaryWriter = SummaryWriter
                except ImportError:
                    self._SummaryWriter = None
        else:
            self._SummaryWriter = None
        self.tb_writers = tb_writers

    def _init_summary_writer(self, args, log_dir=None):
        log_dir = log_dir or args.logging_dir
        if self._SummaryWriter is not None:
            self.tb_writers = dict(train=self._SummaryWriter(log_dir=os.path.join(log_dir, 'train')),
                                   eval=self._SummaryWriter(log_dir=os.path.join(log_dir, 'eval')))

    def on_train_begin(self, args, state, control, **kwargs):
        if not state.is_world_process_zero:
            return

        log_dir = None

        if state.is_hyper_param_search:
            trial_name = state.trial_name
            if trial_name is not None:
                log_dir = os.path.join(args.logging_dir, trial_name)

        if self.tb_writers is None:
            self._init_summary_writer(args, log_dir)

        for k, tbw in self.tb_writers.items():
            tbw.add_text("args", args.to_json_string())
            if "model" in kwargs:
                model = kwargs["model"]
                if hasattr(model, "config") and model.config is not None:
                    model_config_json = model.config.to_json_string()
                    tbw.add_text("model_config", model_config_json)
            # Version of TensorBoard coming from tensorboardX does not have this method.
            if hasattr(tbw, "add_hparams"):
                tbw.add_hparams(args.to_sanitized_dict(), metric_dict={})

    def on_log(self, args, state, control, logs=None, **kwargs):
        if not state.is_world_process_zero:
            return

        if self.tb_writers is None:
            self._init_summary_writer(args)

        for tbk, tbw in self.tb_writers.items():
            logs_new = custom_rewrite_logs(logs, mode=tbk)
            for k, v in logs_new.items():
                if isinstance(v, (int, float)):
                    tbw.add_scalar(k, v, state.global_step)
                else:
                    logger.warning(
                        "Trainer is attempting to log a value of "
                        f'"{v}" of type {type(v)} for key "{k}" as a scalar. '
                        "This invocation of Tensorboard's writer.add_scalar() "
                        "is incorrect so we dropped this attribute."
                    )
            tbw.flush()

    def on_train_end(self, args, state, control, **kwargs):
        for tbw in self.tb_writers.values():
            tbw.close()
        self.tb_writers = None

In [ ]:
#trainer.remove_callback(TensorBoardCallback)

In [ ]:
import torch

for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

odict_keys(['loss', 'start_logits', 'end_logits'])

In [ ]:
output.start_logits.shape, output.end_logits.shape

(torch.Size([1, 384]), torch.Size([1, 384]))

In [ ]:
output.start_logits.argmax(dim=-1), output.end_logits.argmax(dim=-1)

(tensor([0], device='cuda:0'), tensor([0], device='cuda:0'))

In [ ]:
n_best_size = 20

In [ ]:
import numpy as np

start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": "" # We need to find a way to get back the original substring corresponding to the answer in the context
                }
            )

In [ ]:
def prepare_validation_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [ ]:
validation_features = datasets["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets["validation"].column_names
)

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [ ]:
raw_predictions = trainer.predict(validation_features)

In [ ]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

In [ ]:
max_answer_length = 30

In [ ]:
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]
# The first feature comes from the first example. For the more general case, we will need to be match the example_id to
# an example index
context = datasets["validation"][0]["context"]

# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
        # to part of the input_ids that are not in the context.
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers

[{'score': -7.6822615,
  'text': 'More than 40 pilots currently volunteer for Project Wingman'},
 {'score': -8.2574005,
  'text': 'More than 40 pilots currently volunteer for Project Wing'},
 {'score': -8.360842, 'text': 'More than 40 pilots'},
 {'score': -8.392422,
  'text': 'More than 40 pilots currently volunteer for Project'},
 {'score': -10.306692,
  'text': 'The U.K. and European Union require such peer-to-peer programs, and the FAA recommends them for all airlines.'},
 {'score': -10.42984,
  'text': 'U.K. and European Union require such peer-to-peer programs, and the FAA recommends them for all airlines.'},
 {'score': -10.502052, 'text': '.'},
 {'score': -10.562281,
  'text': '. The U.K. and European Union require such peer-to-peer programs, and the FAA recommends them for all airlines.'},
 {'score': -10.587524,
  'text': 'and the FAA recommends them for all airlines.'},
 {'score': -11.567437, 'text': '7,000'},
 {'score': -12.017357, 'text': '7,000 pilots'},
 {'score': -12.14489

In [ ]:
datasets["validation"][0]["answers"]

"{'answer_start': [-1], 'text': ['No answer exists']}"

In [ ]:
import collections

examples = datasets["validation"]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

In [ ]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []
        
        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}
        
        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions

In [ ]:
final_predictions = postprocess_qa_predictions(datasets["validation"], validation_features, raw_predictions.predictions)

Post-processing 1 example predictions split into 1 features.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
final_predictions

OrderedDict([(1,
              'More than 40 pilots currently volunteer for Project Wingman')])

In [ ]:
#pd.DataFrame.from_dict(final_predictions, orient="index").to_csv("prediction.csv")